<h2>FINAL DATA PREPROCESSING & MODEL TRAINING</h2>

Script ini digunakan untuk mempreprocess data menjadi data final yang dapat digunakan untuk training model, dan training model LSTM itu sendiri. Secara umum, script ini meliputi:
<h4>Data Preprocessing</h4>
- Padding & Truncating data, pada dasarnya memastikan seluruh trj_id memiliki jumlah data yang sama, yakni 20 data, dengan menambahkan data yang kurang dengan 0 dan memotong data yang kelebihan dan mengambil 20 data paling belakang.
- Membagi data menjadi X (koordinat, keceptan, dll yang dimasukkan ke model untuk melatih model) dan y (koordinat yang benar untuk dibandingkan dengan koordinat hasil prediksi model). y akan diambil dari koordinat terakhir tiap trajectory.
- Membagi data menjadi training data dan testing data, dengan pembagian 90%/10%.
<br/>
<h4>Model Training</h4>
- Mendefinisikan dan melatih model LSTM berdasarkan data yang telah siap.
- Arsitektur, epoch, loss, optimizer, dan dll dapat dengan bebas diubah-ubah untuk mencoba model lain.
<br/>
Jika ingin melihat performa model pertama yang telah saya latih, maka model itu sudah disave dalam folder models dengan nama model_v1.keras. Model dapat ditest menggunakan script Model Test.ipynb

In [1]:
## Import libraries
import pandas as pd
import numpy as np
import xarray as xr
import tensorflow as tf
import plotly.express as px
from sklearn.model_selection import train_test_split
import datetime

In [2]:
# Load Tensorboard
%load_ext tensorboard

In [3]:
## Read data from csv
resampled_data = pd.read_csv('../clean_data.csv', index_col='trj_id').drop('Unnamed: 1', axis=1)
print(resampled_data)

          rawlat      rawlng      speed     bearing  hour_of_day  day_of_week
trj_id                                                                       
1      -6.198042  106.769008   4.322800  179.920000           14            3
1      -6.200972  106.769202   8.014167  173.233333           14            3
1      -6.205394  106.769768  10.116136  171.477273           14            3
1      -6.210496  106.771217   9.307667  156.683333           14            3
1      -6.214969  106.773830  10.103333  139.777778           14            3
...          ...         ...        ...         ...          ...          ...
9999   -6.187751  106.845707  10.584667  329.600000            4            6
9999   -6.184123  106.843546   4.508780  324.512195            4            6
9999   -6.182706  106.842869   2.776724  287.137931            4            6
9999   -6.180504  106.842337   5.244333  326.850000            4            6
9999   -6.179029  106.841998   2.330952  231.095238            4

In [4]:
# Pad and truncate the timestamps in the dataframe
# Ini buat kita samain input modelnya, gw potong timestampnya jadi pasti ada 20 timestamp per sample. Kalo lebih dipotong, kalo kurang ditambahin 0 di depannya

def pad_truncate_dataframe(df, max_len, padding='pre', truncating='post', pad_value=0):
  # Split the dataframe by samples (first level of multi-index)
  samples = df.groupby(level=0)

  # Define a function to pad/truncate a single sample
  def pad_truncate_sample(sample):
    # Extract the values from a single sample
    values = sample.values

    # Pad/truncate automatically using pad_sequences
    padded_truncated = tf.keras.preprocessing.sequence.pad_sequences(
        [values], maxlen=max_len, padding=padding, truncating=truncating, value=pad_value, dtype='float64'
    )[0]

    # Convert back to pandas dataframe
    return pd.DataFrame(padded_truncated, columns=sample.columns)

  # Apply the function to each sample and recreate the multi-index dataframe
  padded_df = samples.apply(pad_truncate_sample)
  return padded_df

df_new = pad_truncate_dataframe(resampled_data, 20)

In [5]:
## Convert the multi index dataframe to a numpy 3D array for better integration to TensorFlow (samples, timesteps, features)
numpy_data = df_new.to_xarray().to_array().to_numpy()
numpy_data = np.transpose(numpy_data, (1, 2, 0))
numpy_data.shape # Should be (55994, 20, 6) for 55994 samples, 20 timesteps, and 6 features

(55994, 20, 6)

In [6]:
## Split the data to x (feature values) and y (target values)
x_data = numpy_data[:, :-1, :]
y_data = numpy_data[:, -1, 0:2]

In [7]:
# Split the data to train and test splits (train to train the model, test to test the model on new data after trained)
test_size = 0.1

x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=test_size, random_state=42)

In [8]:
## Define normalization layer
normalize_layer = tf.keras.layers.Normalization(axis=-1, input_shape=(19, 6))
normalize_layer.adapt(numpy_data)

c:\Users\adrie\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\layers\preprocessing\normalization.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [9]:
## Get Mean and Variance of Normalization layer to use for normalizing y and denormalizing later
normalize_weights = normalize_layer.get_weights()

mean = normalize_weights[0][0:2]
variance = normalize_weights[1][0:2]

print(mean)
print(variance)

[-5.3450627 91.3314   ]
[   4.8553214 1414.9473   ]


In [10]:
## Normalize y
normalize_y = tf.keras.layers.Normalization(mean=mean, variance=variance)

y_train = normalize_y(y_train)
y_test = normalize_y(y_test)

In [11]:
## Save test data for model testing
np.save('../x_test.npy', x_test)
np.save('../y_test.npy', y_test)

In [17]:
# Define the model
num_features = 6
batch_size = 32

model = tf.keras.Sequential([
  normalize_layer,
  tf.keras.layers.LSTM(64, return_sequences=True),
  tf.keras.layers.LSTM(64, return_sequences=True),
  tf.keras.layers.LSTM(32),
  tf.keras.layers.Dense(2)
])


# Define optimizer
opt = tf.keras.optimizers.Adam(learning_rate=1e-4)

model.compile(loss='mse', optimizer=opt, metrics=['mae'])

In [18]:
log_dir = "../logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', mode='min', patience=20)

lr_schedule = tf.keras.callbacks.LearningRateScheduler(lambda epoch: 1e-4 * 10 ** (epoch/20))

model.fit(
  x=x_train, 
  y=y_train,
  epochs=100,
  batch_size=64,
  validation_data=(x_test, y_test),
  callbacks=[tensorboard_callback, early_stopping, lr_schedule])

model.evaluate(x_test, y_test)

Epoch 1/100
788/788 ━━━━━━━━━━━━━━━━━━━━ 21s 22ms/step - loss: 0.0327 - mae: 0.1096 - val_loss: 5.4749e-04 - val_mae: 0.0160 - learning_rate: 1.0000e-04
Epoch 2/100
788/788 ━━━━━━━━━━━━━━━━━━━━ 18s 23ms/step - loss: 4.3306e-04 - mae: 0.0140 - val_loss: 1.9788e-04 - val_mae: 0.0087 - learning_rate: 1.1220e-04
Epoch 3/100
788/788 ━━━━━━━━━━━━━━━━━━━━ 19s 24ms/step - loss: 1.8163e-04 - mae: 0.0084 - val_loss: 1.2874e-04 - val_mae: 0.0069 - learning_rate: 1.2589e-04
Epoch 4/100
788/788 ━━━━━━━━━━━━━━━━━━━━ 22s 28ms/step - loss: 1.3015e-04 - mae: 0.0070 - val_loss: 9.4039e-05 - val_mae: 0.0060 - learning_rate: 1.4125e-04
Epoch 5/100
788/788 ━━━━━━━━━━━━━━━━━━━━ 21s 27ms/step - loss: 9.7780e-05 - mae: 0.0060 - val_loss: 7.0074e-05 - val_mae: 0.0052 - learning_rate: 1.5849e-04
Epoch 6/100
788/788 ━━━━━━━━━━━━━━━━━━━━ 19s 24ms/step - loss: 8.7450e-05 - mae: 0.0057 - val_loss: 6.0882e-05 - val_mae: 0.0049 - learning_rate: 1.7783e-04
Epoch 7/100
788/788 ━━━━━━━━━━━━━━━━━━━━ 19s 24ms/step - loss:

KeyboardInterrupt: 

In [ ]:
## Save the model
model.save('../models/model_v1.keras')